In [ ]:
import json
from collections import defaultdict
from tqdm.auto import tqdm

In [ ]:
data = json.load(open('../../../data/ConceptNet/test.json', 'r'))

In [ ]:
relation_wise_examples = defaultdict(list)
relation_wise_subjects = defaultdict(set)
for example in tqdm(data):
    if example['rel_id'] in ['Synonym', 'Antonym', 'HasProperty', 'NotHasProperty', 'Desires', 'NotDesires', 'CapableOf', 'NotCapableOf']:
        relation_wise_examples[example['rel_id']].append(example)
        relation_wise_subjects[example['rel_id']].add(example['subj'])

In [ ]:
all_data = json.load(open('../../../data/ConceptNet/all.json', 'r'))

In [ ]:
all_relation_wise_examples = defaultdict(list)
all_relation_wise_subjects = defaultdict(set)
for example in tqdm(all_data):
    if example['rel_id'] in ['Synonym', 'Antonym', 'HasProperty', 'NotHasProperty', 'Desires', 'NotDesires', 'CapableOf', 'NotCapableOf']:
        all_relation_wise_examples[example['rel_id']].append(example)
        all_relation_wise_subjects[example['rel_id']].add(example['subj'])

In [ ]:
for rel_id in relation_wise_examples:
    print(rel_id, len(relation_wise_examples[rel_id]))

In [ ]:
filtered_relation_wise_examples = defaultdict(list)
merged_relation_wise_subjects = {}
merged_relation_wise_subjects['Synonym'] = relation_wise_subjects['Synonym'] & all_relation_wise_subjects['Antonym']
merged_relation_wise_subjects['Antonym'] = relation_wise_subjects['Antonym'] & all_relation_wise_subjects['Synonym']
merged_relation_wise_subjects['HasProperty'] = relation_wise_subjects['HasProperty'] & all_relation_wise_subjects['NotHasProperty']
merged_relation_wise_subjects['NotHasProperty'] = relation_wise_subjects['NotHasProperty'] & all_relation_wise_subjects['HasProperty']
merged_relation_wise_subjects['Desires'] = relation_wise_subjects['Desires'] & all_relation_wise_subjects['NotDesires']
merged_relation_wise_subjects['NotDesires'] = relation_wise_subjects['NotDesires'] & all_relation_wise_subjects['Desires']
merged_relation_wise_subjects['CapableOf'] = relation_wise_subjects['CapableOf'] & all_relation_wise_subjects['NotCapableOf']
merged_relation_wise_subjects['NotCapableOf'] = relation_wise_subjects['NotCapableOf'] & all_relation_wise_subjects['CapableOf']

In [ ]:
for rel_id in relation_wise_examples:
    for example in relation_wise_examples[rel_id]:
        if example['subj'] in merged_relation_wise_subjects[rel_id]:
            filtered_relation_wise_examples[rel_id].append(example)

In [ ]:
for rel_id in filtered_relation_wise_examples:
    print(rel_id, len(filtered_relation_wise_examples[rel_id]))

In [ ]:
opposite_relation_ids = []
for example in filtered_relation_wise_examples['Synonym']:
    opposite_relation_ids.append(example['uid'])
for example in filtered_relation_wise_examples['Antonym']:
    opposite_relation_ids.append(example['uid'])

In [ ]:
synonym_answers = defaultdict(list)
antonym_answers = defaultdict(list)
for example in all_relation_wise_examples['Synonym']:
    if example['subj'] in relation_wise_subjects['Antonym']:
        synonym_answers[example['subj']].append(example['output'])
for example in all_relation_wise_examples['Antonym']:
    if example['subj'] in relation_wise_subjects['Synonym']:
        antonym_answers[example['subj']].append(example['output'])

In [ ]:
with open('opposite_relation_ids.json', 'w') as fout:
    json.dump(opposite_relation_ids, fout)
with open('synonym_answers.json', 'w') as fout:
    json.dump(synonym_answers, fout)
with open('antonym_answers.json', 'w') as fout:
    json.dump(antonym_answers, fout)